# CVXPY MIP
https://www.cvxpy.org/examples/

In [2]:
!pip install cvxpy

     -------------------------------------- 72.0/72.0 kB 996.6 kB/s eta 0:00:00

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.3.0 requires daal==2021.2.3, which is not installed.
numba 0.54.1 requires numpy<1.21,>=1.17, but you have numpy 1.22.4 which is incompatible.



     --------------------------------------- 14.7/14.7 MB 12.6 MB/s eta 0:00:00
   ---------------------------------------- 1.0/1.0 MB 11.0 MB/s eta 0:00:00
   --------------------------------------- 355.1/355.1 kB 11.1 MB/s eta 0:00:00
   ---------------------------------------- 292.9/292.9 kB 6.0 MB/s eta 0:00:00
   ---------------------------------------- 8.4/8.4 MB 12.5 MB/s eta 0:00:00
   ---------------------------------------- 227.7/227.7 kB 6.8 MB/s eta 0:00:00
   ---------------------------------------- 85.1/85.1 kB 1.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.2
    Uninstalling numpy-1.26.2:
      Successfully uninstalled numpy-1.26.2


In [223]:
!pip install gurobipy

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [269]:
%reset -f
import numpy as np
import cvxpy as cp

In [270]:
# Parameters
# Parameters
M = 3; # Number of machines
P = 4; # Number of personnel
T = 5; # Number of tasks
#c = [2.1, 3.0, 1.6]; # Cost of using each machine m per unit time
c = cp.Parameter(M, value=[2.1, 3.0, 1.6], boolean=False); # Cost of using each machine m per unit time
#e = [12.1, 13.0, 14.0, 11.6]; # Cost of employing each person p per unit time
e = cp.Parameter(P, value=[12.1, 13.0, 14.0, 11.6], boolean=False); # Cost of employing each person p per unit time
o = 123.2; # Fixed overhead cost
u = [222.1, 223.0, 1111.6]; # Usage limit (max time over the period) for each machine m
v = [140.0, 141.1, 139.6, 142.9]; # Work hours limit (max time over the period) for each person p
l = [4.02, 5.1, 3.6, 4.29, 6.08]; # Duration of each task t in hours. Assumed to be independent on machine m!
d = [11.0, 11.0, 11.0, 11.0, 10.0]; # Deadline of each task t in hours
r = [22.02, 24.1, 22.6, 23.29, 22.08]; # Tardiness starts at these times for each task t
wc = 1.0; # Weight of cost objective
wt = 1.0; # Weight of tardiness objective

# Decision variables
x = cp.Variable(M, value=np.zeros(M), boolean=False) # Time each machine m is used
y = cp.Variable(P, value=np.zeros(P), boolean=False) # Time each person p is employed
a = cp.Variable((T, M), boolean=True); # Assignment of each task t to each machine m
b = cp.Variable((T, P), boolean=True); # Assignment of each task t to each person m
f = cp.Variable(T, value=np.zeros(T), boolean=False) # Finish time for each task t

###obj = cp.Minimize(sum(np.dot(variable, one_vec)))
# Objective 1: Minimize the total cost of quality control resource allocation.
# Objective 2: Minimize the total tardiness.
obj = cp.Minimize( wc * (np.dot(c, x) + np.dot(e, y) + o) + wt * cp.sum(cp.maximum(0, f - r)))
# No throughput. Assuming all tasks must be completed!

constraints = []

# Each machine has a usage limit:

for m in range(M):
    constraints.append(x[m] <= u[m])

# Each person has a work hours limit:
for p in range(P):
    constraints.append(y[p] <= v[p])

# Each task is assigned to only one machine:
for t in range(T):
    constraints.append(cp.sum(a[t]) == 1)

# Each task is assigned to only one person:
for t in range(T):
    constraints.append(cp.sum(b[t]) == 1)

# Resource Workload. Match task duration with machine and personnel usage:
for m in range(M):
    constraints.append(x[m] == cp.sum(a[:,m]@l))
for p in range(P):
    constraints.append(y[p] == cp.sum(b[:,p]@l))

# Finish time for each task t:
constraints.append(f[0] == l[0]);
#constraints.append(f[1] == cp.sum( a[1,:] * ( cp.maximum(0, a[1,:] + a[0,:] - 1) * l[0] + cp.maximum(0, a[1,:] + a[1,:] - 1) * l[1] ) ))
#constraints.append(f[1] == cp.maximum(0, a[1,:] + a[0,:] - 1) * l[0] + cp.maximum(0, a[1,:] + a[1,:] - 1) * l[1])
#Doesn't follow DCP rules!
for t in range(2,T):
    #Non-convex:
    constraints.append(f[t] == cp.sum( a[t,:] * l[t] ))
    #cp.maximum(0, a[t,:] + a[t1,:] - 1) * l[t1]
    
    #constraints.append(f[1] == cp.sum(l[0]*(a[0,:]==a[1,:])) + l[1]);
    # Assuming tasks are executed in ascending order of its number t on each machine!
    
    # Reformulate to make it linear in a:
    # Truth Table:
    # a[t,m]   a[t1,m]   x
    # 0        0         0
    # 0        1         0
    # 1        0         0
    # 1        1         1
    #constraints.append(f[t] == cp.sum( cp.sum( cp.maximum(0, a[t,:] + a[t1,:] - 1) * l[t1] for t1 in range(t) ), axis=1 ));

# Due Date for each task t:
for t in range(T):
    constraints.append(f[t] <= d[t])
    
problem = cp.Problem(obj, constraints=constraints)

C:\Users\Admin\anaconda3\lib\site-packages\cvxpy\expressions\expression.py:621: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 178 times so far.

  warnings.warn(msg, UserWarning)
C:\Users\Admin\anaconda3\lib\site-packages\cvxpy\expressions\expression.py:621: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 179 times so far.

  warnings.warn(msg, UserWarning)


In [271]:
problem.solve()

DCPError: Problem does not follow DCP rules. Specifically:
The following constraints are not DCP:
var9444[1] == maximum(0.0, var9442[1, 0:3] + var9442[0, 0:3] + Promote(-1.0, (3,))) @ Promote(4.02, (3,)) + maximum(0.0, var9442[1, 0:3] + var9442[1, 0:3] + Promote(-1.0, (3,))) @ Promote(5.1, (3,)) , because the following subexpressions are not:
|--  var9444[1] == maximum(0.0, var9442[1, 0:3] + var9442[0, 0:3] + Promote(-1.0, (3,))) @ Promote(4.02, (3,)) + maximum(0.0, var9442[1, 0:3] + var9442[1, 0:3] + Promote(-1.0, (3,))) @ Promote(5.1, (3,))